In [4]:
import os
import easyocr
import cv2
from matplotlib import pyplot as plt
import numpy as np
import re
from spellchecker import SpellChecker
import argparse
from PIL import Image


reader = easyocr.Reader(['en'])

In [ ]:
# Create an ArgumentParser object
parser = argparse.ArgumentParser(description='Process an image.')

# Define the command-line argument for the image file path
parser.add_argument('--image_path', type=str, help='Path to the image file to process')

# Parse the command-line arguments
args = parser.parse_args()

# Access the image file path
image_path = args.image_path

# Initialize img variable
img = Image.open(image_path)


In [5]:
result = reader.readtext(img, paragraph=True, detail = 0)
result

['Splenda Nutrition Facts 200 servings per container Serving Size packet (1g) Amount per serving Calories o5 DV Total Fat Og 098 Sodlum Orrg 096 Total Carb_ 096 Tolal Sugars Incudac <lg Added Sugars Protein Og daniican acuit Dulnny nluraled E: bn; Folu-aiumt Kaeif cirium Irn nd RDV - #~Daly Vnlue',
 '3 9 2 6 0 U 5',
 'UNGREDIENTS: DEXTROSE, Maltodextrin; SPLENDA " Sucralose, PARVE (GLUTEN FREE HALAL SuitABLE For PEOPLE With DiABETES. STORE IN COOL, Dry PLACE,']

In [6]:
ingredients = ' '.join(result)
ingredients

'Splenda Nutrition Facts 200 servings per container Serving Size packet (1g) Amount per serving Calories o5 DV Total Fat Og 098 Sodlum Orrg 096 Total Carb_ 096 Tolal Sugars Incudac <lg Added Sugars Protein Og daniican acuit Dulnny nluraled E: bn; Folu-aiumt Kaeif cirium Irn nd RDV - #~Daly Vnlue 3 9 2 6 0 U 5 UNGREDIENTS: DEXTROSE, Maltodextrin; SPLENDA " Sucralose, PARVE (GLUTEN FREE HALAL SuitABLE For PEOPLE With DiABETES. STORE IN COOL, Dry PLACE,'

In [7]:
words = re.split(r'[,\'\":(){}.\[\]\=_\;\\]', ingredients)

misread = []
spell = SpellChecker()

for i in range(len(words)):
    words[i] = words[i].strip()
    words[i] = ' '.join(words[i].split())

while ("" in words):
    words.remove("")

for i in range(len(words)):
    if " " in words[i]:
        wordsSet = words[i].split()
        for j in range(len(wordsSet)):
            temp = wordsSet[j]
            if temp != spell.correction(temp):
                temp = spell.correction(temp)
                if (temp is None):
                    temp = wordsSet[j]
            wordsSet[j] = temp
        words[i] = " ".join(wordsSet)


words[len(words)-1] = words[len(words)-1].replace("and ", "")
words[0] = words[0].replace("ingredients", "")
words[0] = words[0].replace("Ingredients", "")
words[0] = words[0].replace("INGREDIENTS", "")

while ("" in words):
    words.remove("")

words

['spend Nutrition Facts 200 servings per container Serving Size packet',
 '1g',
 'Amount per serving Calories of do Total Fat Og 098 sodium org 096 Total Carb',
 '096 total Sugars Incudac all Added Sugars Protein Og daniican acquit dunny nluraled E',
 'bn',
 'Folu-aiumt katie circus Irn and rev - daly value 3 9 2 6 0 U 5 ingredients',
 'DEXTROSE',
 'Maltodextrin',
 'SPLENDA',
 'Sucralose',
 'PARVE',
 'GLUTEN FREE HALAL SuitABLE For PEOPLE With DiABETES',
 'STORE IN COOL',
 'Dry PLACE']

In [8]:
carcinogens = ["butylated hydroxyanisole", "potassium bromate", "acrymalide", "sodium nitrate", "sodium nitrite", "propylene oxide", "yellow 6", "yellow #6", "red 6", "red #6"]
carcSimilar = []
for i in range (len(words)):
    for j in range (len(carcinogens)):
        if carcinogens[j].lower() in words[i].lower():
            carcSimilar.append(carcinogens[j])
carcSimilar

[]

In [9]:
hiddenSugars = ["aspartame", "sucralose", "acesulfame potassium", "saccharin", "neotame", "sorbitol", "xylitol", "erythritol", "maltitol", "maltodextrin"]
sugarsSimilar = []
for i in range (len(words)):
    for j in range (len(hiddenSugars)):
        if hiddenSugars[j].lower() in words[i].lower():
            sugarsSimilar.append(hiddenSugars[j])
sugarsSimilar

['maltodextrin', 'sucralose']